In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import pandas as pd
import re
import random
import numpy as np
import torch
from transformers import AutoTokenizer

In [3]:
test_1 = pd.read_csv('/content/drive/MyDrive/텍스트계약서/summary_cosmetic_1.csv')

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


KeyboardInterrupt: 

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 모델과 토크나이저 로딩
model_name = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,  # bfloat16 사용
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

save_dir = '/content/drive/MyDrive/models/exone_1'
# 저장 예시
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

# 로딩 예시
loaded_model = AutoModelForCausalLM.from_pretrained(save_dir,trust_remote_code=True)
loaded_tokenizer = AutoTokenizer.from_pretrained(save_dir,trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
def generate_response(sentence):
  prompt =  f"""
  원본 문장:{sentence}
원본 문장의 맥락을 살펴보고, 빠르게 문장을 요약하여 재구성합니다.
제목은 그대로 두시고, 내용의 핵심을 추출하여 전체적으로 요약하면 됩니다.
결과는 하나의 문장으로 표현하면 됩니다.
"""

  messages = [
    {"role": "system",
     "content": "You are an excellent sentence summarizer. You understand the context and concisely summarize key sentences as an assistant."},
    {"role": "user", "content": prompt}
  ]
  input_ids = loaded_tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
  )

  output = loaded_model.generate(
    input_ids.to("cuda"),
    eos_token_id=loaded_tokenizer.eos_token_id,
    max_new_tokens=400,
    do_sample=False
)
  generated_text = loaded_tokenizer.decode(output[0], skip_special_tokens=True)

    # 프롬프트 길이를 기준으로 생성된 부분만 추출
  prompt_length = input_ids.shape[1]  # 프롬프트 길이
  generated_text_only = generated_text[len(loaded_tokenizer.decode(input_ids[0], skip_special_tokens=True)):]

  return generated_text_only.strip()  # 양 끝 공백 제거

In [14]:
def generate_response(sentence):
    prompt = f"""
    원본 문장:{sentence}
    원본 문장의 맥락을 살펴보고, 빠르게 문장을 요약하여 재구성합니다.
    제목은 그대로 두시고, 내용의 핵심을 추출하여 전체적으로 요약하면 됩니다.
    결과는 하나의 문장으로 표현하면 됩니다.
    """

    messages = [
        {"role": "system",
         "content": "You are an excellent sentence summarizer. You understand the context and concisely summarize key sentences as an assistant."},
        {"role": "user", "content": prompt}
    ]

    # 장치 선택 (CUDA가 가능하면 GPU, 그렇지 않으면 CPU)
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # 텍스트를 토크나이즈하고 텐서를 생성
    input_ids = loaded_tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    # input_ids 텐서를 올바른 장치로 이동
    input_ids = input_ids.to(device)

    # 모델을 사용하여 텍스트 생성
    output = loaded_model.generate(
        input_ids,
        eos_token_id=loaded_tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False
    )

    generated_text = loaded_tokenizer.decode(output[0], skip_special_tokens=True)

    # 생성된 텍스트에서 프롬프트 길이를 기준으로 텍스트만 추출
    prompt_length = input_ids.shape[1]  # 프롬프트 길이
    generated_text_only = generated_text[len(loaded_tokenizer.decode(input_ids[0], skip_special_tokens=True)):]

    return generated_text_only.strip()  # 양 끝 공백 제거


In [7]:
sentence = """
제7조 [상품의 반품]  ① “갑”은 정당한 사유 없이 “을”로부터 매입한 상품의 전부 또는 일부를  “을”에게 반품(매입한 상품을 다른 상품과 바꾸는 등 실질적으로 반품이 되는  경우를 포함한다)할 수 없다. 다만, 다음 각 호의 어느 하나에 해당하는 경우로  서 거래관행상 합리적이라고 인정되는 기간 안에 반품하는 경우에는 정당한 사 유가 있는 것으로 추정한다.   1. 납품받은 상품에 “을”의 책임 있는 사유로 인한 오손, 훼손 또는 하자가  있는 경우   2. 납품받은 상품이 주문한 상품과 다른 경우   3. 납품받은 상품을 반품함으로써 생기는 손실을 “갑”이 부담하고 “을”의  사전 동의를 받은 경우   4. '프리미엄 시즌 한정 제품' 및 '시즌별 스페셜 컬렉션' 등 일정한 기간이나  계절에 집중적으로 판매되는 화장품 상품에 대하여 다음 조건에 따라 반품 하는 경우 입고일로부터 60일 이내 총 수량의 10% 이내 서울 강남구 테헤란로33길 3-6 역삼스마트타워 상품 하자:“을”, 기타 사유:“갑”   5. 신ㆍ구 상품의 교체, 시장테스트 상품의 반품 등 “을”이 자기에게 직접적 으로 이익이 된다고 판단하여 반품일 이전에 자발적으로 반품을 서면으로 요 청한 경우. 이 경우, “을”은 해당 반품이 자기에게 직접적으로 이익이 된 다는 객관적인 근거자료를 “갑”에게 제출한다.   6. 납품받은 상품이 위조물이거나 타인의 지식재산권을 침해하는 등으로 법령  또는 행정 지침을 위반하여 적법한 상품판매가 불가능한 경우   7. 수입 명품 브랜드 등 해당 브랜드 정책에 따라 반품이 이루어지고, “갑”이  “을”에게 반품 관련 서류를 요구하거나 수취하기 어려운 특별한 사정이 있 는 경우   8. 제1호부터 제7호까지의 규정에 준하는 정당한 사유가 있는 경우  ② 다음 각 호의 어느 하나에 해당하는 경우와 같이 사회 통념상 “을”이 반품을  요청할 가능성이 거의 없는 경우에는 “을”이 제1항 제5호에 따라 반품 요청을  하였다고 하더라도 특별한 사정이 없는 한 “을”에게 반품할 수 없다.   1. “갑”이 전시 등에 사용하여 상품 가치가 현저히 훼손된 상품에 대한 반품.   (단, 납품업자가 고급 브랜드 이미지 유지 등을 위해 할인판매를 원하지 않 아 자발적으로 반품을 요청한 경우는 제외)   2. PB상품 등 “갑”만을 위해 생산하여 납품하거나 “갑”의 상표권을 부착하여  생산한 제품으로 “갑”이외의 다른 유통채널로 판매가 어려운 상품에 대한  반품   3. “갑”이 “을”과 협의 없이 점포 또는 매장을 리뉴얼하여 발생한 재고  처리를 위한 반품 등 “갑”의 귀책사유로 인한 반품      ③ 제1항제1호 및 제2호에 따른 반품 기간은 납품일로부터 ()일 이내로 한다. 다 만, 외관상 즉시 확인할 수 없는 하자는 객관적 자료(공인 검사 성적서 등)를 제 출하면 상호 협의로 기한을 연장할 수 있다.   반품 요청은 서면으로 제출해야 하며, 갑은 정당한 사유 없이 반품할 수 없고,  을에게 문제 해결 기회를 먼저 부여해야 한다. 동일 사유로 반복 반품 시, 양측 이 원인 분석 및 협의를 진행하며, 갑이 부당한 반품을 지속하면 을은 거부할 수  있다.  ④ 제3항에도 불구하고, 제1항제1호 및 제2호에 따른 반품은 납품 후 검수 및 매 입 완료 전(최대 2일)까지 요청해야 한다.   단, 육안으로 확인이 어려운 하자는 검사 결과 제출 시 조정 가능하며, 갑은 정 당한 사유 없이 반품할 수 없다. 을에게 보완 기회를 우선 제공하며, 부당 반품 이 지속되면 거부할 수 있다.  ⑤ “갑”은 반품에 대한 기준과 절차를 사전에 자신의 홈페이지 등 “을”이 상 시 접근 가능한 매체수단에 공지하거나, 계약체결 시 별도의 서면으로 “을”에 게 통지한다. 또한 동 기준이 변경되었을 경우에도 같은 방법으로 “을”에게  통지한다.
"""

In [15]:
result= generate_response(sentence)
print(result)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
test_1['input']

,input
0,제1조 [목적] 이 계약서의 작성 목적은 “갑”과 “을” 간 상품의 직매입거래(“...
1,제2조 [기본원칙] ① “갑”과 “을”은 상호이익을 존중하고 이 계약상 의무를 신...
2,제3조 [공정거래 준수 및 동반성장 지원] ① “갑”과 “을”은 공정하고 투명한 ...
3,"제4조 [상품의 납품] ① “을”이 “갑”에게 납품하여야 하는 상품의 종류, 수량..."
4,제5조 [검수기준 및 품질검사] ① “갑”은 납품받는 상품에 관하여 “갑”의 검수...
5,제6조 [납품대금 지급 및 감액금지] ① 갑은 납품대금을 상품 입고일로부터 최대 ...
6,제6조의2 [공급원가 변동에 따른 납품 가격의 조정] ①을은 계약 체결 이후 최...
7,제7조 [상품의 반품] ① “갑”은 정당한 사유 없이 “을”로부터 매입한 상품의 ...
8,제8조 [판매장려금] ① “갑”은 상품의 판매촉진을 위하여 다음 조건에 따라 “...
9,제9조 [판촉사원 파견 등] ① “갑”은 “을”로부터 “을”의 종업원이나 “을”이...


In [12]:
# test_1['input']에 generate_response 함수를 순차적으로 적용하여 결과를 'result' 컬럼에 저장
test_1['result'] = test_1['input'].apply(generate_response)

In [ ]:
test_1.to_csv('test_summ.csv', encoding='utf-8-sig', index=False)